# COMP30760 Assignment 2 - Task 1

In this Assignment we will collect stock price data from HTML pages and construct time series from the data.

For this Assignment I have decided to collect stock frice data from the following companies:

Apple Inc. (AAPL)

Alphabet Inc. (GOOG)

Ebay (EBAY)

Pfizer (PFE)

This Notebook covers Task1-Data Collection and Preperation. I've decided to save the preprocessed versions of the data as csv files.

Below are some useful imports which we will use in this Notebook.

In [1]:
import urllib.request
import pandas as pd
import bs4
from pathlib import Path

Below are some settings for data collection purposes

In [2]:
#base link of html page
base_link="http://mlg.ucd.ie/modules/COMP30760/stocks/data-"
#html link for each stock
stock_link={"AAPL":"aapl.html","GOOG":"goog.html","EBAY":"ebay.html","PFE":"pfe.html"}
#ticker associated with each stock
stock_ticker={"Apple":"AAPL","Google":"GOOG","Ebay":"EBAY","Pfizer":"PFE"}
#create names for column
column_names=["Open","High","Low","Close"]

Create directory for raw data storage, if it does not already exist:

In [3]:
dir_raw = Path("raw_data_assignment_2")
dir_raw.mkdir(parents=True, exist_ok=True)

## Data Collection

Lets define a function to fetch the raw data from the html website:

In [4]:
def fetch_data(link, stock):
    #fetching the page where the data is stroed by adding the link of the stock to the base link
    response = urllib.request.urlopen(base_link+link)
    print ("Fetching stock data for "+stock+" .....")
    #raw html data
    html_data = response.read().decode()
    
    return html_data

Lets also define a function to count how many rows in our stock data:

In [5]:

def count_rows(parser):
    count=0
    row=0
    for match in parser.find_all("td"):
        text = match.get_text()
        count+=1
        if count == 8:
            row+=1
            count=0
    print("Rows: %i"%row)
            
    return row

## Data Preprocessing
Lets now create a function to parse the raw data and store it in a 2d array so we can easily convert it to a dataframe.

In [6]:
def parse_raw_data(data,stock): 
    print("Parsing "+stock+" data.........")
    parser = bs4.BeautifulSoup(data,"html.parser")
    count=0
    row=0
    #creating our 2d array (may be an unconventional way of doing this but worked for me :))
    #count rows function being used here also to calculate how many rows there are
    data=[[] for _ in range(count_rows(parser))]
    #for loop iterates through each line and gets all relevant data
    #From looking at the raw html data I know that inside td is where all 
    #the relevant data lies
    for match in parser.find_all("td"):
        text = match.get_text()
        data[row].append(text)
        count+=1
        #from the raw html file we I know that each row has 8 columns so we iterate onto the next column to be filled
        if count == 8:
            row+=1
            count=0
    print("Sucessfully Parsed "+stock+" data")
    
    pdData=pd.DataFrame(data,columns = ['Stock', 'Year','Month','Day', 'Open', 'High','Low', 'Close'] )
    
    return pdData

Now lets create a function to call all of the functions together so it looks nice and clean in a for loop

In [7]:
def fetch_and_parse(stock,link):
    raw_data=fetch_data(link, stock)
    parsed_data=parse_raw_data(raw_data,stock)
    
    return parsed_data
    

Now lets store the created dataframes for each stock in a dictionary to allow for easy accessibility

In [8]:
df_stocks={}

for stock in stock_ticker:
    print("--%s"%stock)
    #create temp variable to store dataframe 
    df_stock=fetch_and_parse(stock_ticker[stock],stock_link[stock_ticker[stock]])
    #store dataframe in given dictionary
    df_stocks[stock]=df_stock
    

--Apple
Fetching stock data for AAPL .....
Parsing AAPL data.........
Rows: 508
Sucessfully Parsed AAPL data
--Google
Fetching stock data for GOOG .....
Parsing GOOG data.........
Rows: 508
Sucessfully Parsed GOOG data
--Ebay
Fetching stock data for EBAY .....
Parsing EBAY data.........
Rows: 508
Sucessfully Parsed EBAY data
--Pfizer
Fetching stock data for PFE .....
Parsing PFE data.........
Rows: 508
Sucessfully Parsed PFE data


Now lets see if our above functions worked...

In [9]:
df_stocks["Google"].head(20)


,Stock,Year,Month,Day,Open,High,Low,Close
0,Stock,Year,Month,Day,Open,High,Low,Close
1,GOOG,2019,10,01,1219.000000,1231.229980,1203.579956,1205.099976
2,GOOG,2019,10,02,1196.979980,1196.979980,1171.290039,1176.630005
3,GOOG,2019,10,03,1180.000000,1189.060059,1162.430054,1187.829956
4,GOOG,2019,10,04,1191.890015,1211.439941,1189.170044,1209.000000
5,GOOG,2019,10,07,1204.400024,1218.203979,1203.750000,1207.680054
6,GOOG,2019,10,08,1197.589966,1206.079956,1189.010010,1189.130005
7,GOOG,2019,10,09,1199.349976,1208.349976,1197.630005,1202.310059
8,GOOG,2019,10,10,1198.579956,1215.000000,1197.339966,1208.670044
9,GOOG,2019,10,11,1222.209961,1228.390015,1213.739990,1215.449951


## Data Clean Up

Lets now clean up the data, as we can see some rows contain data we do not need and also there are some missing values.

Lets Define a funtion first to delete any unwanted rows.


In [10]:
def clear_rows(df):
    #for loop to iterate through each row in dataframe
    for i,check in enumerate(df['Close']):
        if check == "Close":
            print("row "+str(i)+" deleted")
            df=df.drop(i)
        
    df=df.reset_index(drop=True)
    
    
    return df

In [11]:
for stock in df_stocks:
    print(stock+"-----")
    df_stocks[stock]=clear_rows(df_stocks[stock])
    

Apple-----
row 0 deleted
row 65 deleted
row 319 deleted
Google-----
row 0 deleted
row 65 deleted
row 319 deleted
Ebay-----
row 0 deleted
row 65 deleted
row 319 deleted
Pfizer-----
row 0 deleted
row 65 deleted
row 319 deleted


In [12]:
df_stocks["Pfizer"].tail(20)

,Stock,Year,Month,Day,Open,High,Low,Close
485,PFE,2021,09,02,46.480000,46.869999,46.130001,46.840000
486,PFE,2021,09,03,46.900002,47.029999,46.340000,46.840000
487,PFE,2021,09,07,46.990002,47.500000,46.540001,46.750000
488,PFE,2021,09,08,46.509998,46.790001,46.250000,46.509998
489,PFE,2021,09,09,46.470001,46.720001,45.849998,46.029999
490,PFE,2021,09,10,46.270000,46.349998,45.509998,45.590000
491,PFE,2021,09,13,45.740002,45.830002,44.200001,44.580002
492,PFE,2021,09,14,44.669998,45.259998,44.549999,44.709999
493,PFE,2021,09,15,44.560001,45.369999,44.470001,44.770000
494,PFE,2021,09,16,44.880001,44.990002,44.029999,44.470001


Next lets create a function to fill in any missing values. For filling in missing values i am going to take the value above and below and get the average. In some cases there may be cases where there are two successive values missing. in this case I will get the average from two rows below.

In [13]:
def fill_missing_data(stock,stock_name):
    count=0
    #nested for loop to iterate over each value in each column
    for col_name in column_names:
        for i,check in enumerate(stock[col_name]):
            #case for when the first value in the column is null
            if check=="NaN" and i==0:
                stock[col_name][i]=stock[col_name][i+1]
                count+=1
                #case for only one null entry value
            elif check=="NaN":
                #case for when there is two successive values of null entries
                if (stock[col_name][i+1]=="NaN"):
                    stock[col_name][i]=(float(stock[col_name][i-1])+float(stock[col_name][i+2]))/2
                    count+=1
                else:
                    #case for only one null entry value
                    stock[col_name][i]=(float(stock[col_name][i-1])+float(stock[col_name][i+1]))/2
                    count+=1

    print(str(count) +" values filled in "+stock_name+ " dataframe")
    
            
    return stock

In [14]:
for stock in df_stocks:
    df_stocks[stock]=fill_missing_data(df_stocks[stock],stock)

78 values filled in Apple dataframe
73 values filled in Google dataframe
70 values filled in Ebay dataframe
82 values filled in Pfizer dataframe


Lets see if thus function worked

In [15]:
df_stocks["Pfizer"].head(50)

,Stock,Year,Month,Day,Open,High,Low,Close
0,PFE,2019,10,01,33.453510,33.462997,32.884251,32.922199
1,PFE,2019,10,02,33.453510,33.462997,32.884251,32.922199
2,PFE,2019,10,03,32.931690,33.652752,32.827324,33.633778
3,PFE,2019,10,04,33.671726,34.136623,33.633778,34.089184
4,PFE,2019,10,07,33.984821,34.402279,33.965843,33.994308
5,PFE,2019,10,08,33.823528,34.041744,33.605312,33.614799
6,PFE,2019,10,09,33.823528,34.051231,33.709679,33.861481
7,PFE,2019,10,10,33.842506,34.165085,33.776093,33.956356
8,PFE,2019,10,11,34.288425,34.705883,34.222012,34.259960
9,PFE,2019,10,14,34.307400,34.487667,34.136623,34.440228


I am now going to change some of the dtypes in this data frame and also change the index. Most importantly I am going to create a new column called date and merge the da, month and year columns together and change this to dtype(datetime64) and finally set this as the new index. This will help for getting quarterly and monthly data in task 2. I will also change the dtypes for some of the columns as we can see that they are objects  at the moment.

Lets create a function to change these columns.

In [16]:
def change_column(stock):
    #change the Year column to the new date column
    df_stocks[stock]=df_stocks[stock].rename(columns={"Year":"Date"})
    #Assign the date columns with the day month and year values, this makes it easier for changing its dtype
    df_stocks[stock]["Date"]=df_stocks[stock]["Date"].astype(str) + "-"+ df_stocks[stock]["Month"].astype(str)+ "-"+df_stocks[stock]["Day"].astype(str)
    #delete the unwanted columns
    df_stocks[stock] = df_stocks[stock].drop(df_stocks[stock].columns[[2, 3]], axis=1) 
    #change the date dtype to datetime
    df_stocks[stock]['Date']=pd.to_datetime(df_stocks[stock]['Date'])
    #change the dtypes for stock, open, close, high and low
    df_stocks[stock]['Stock']=df_stocks[stock]['Stock'].astype(str)
    df_stocks[stock]['Close']=df_stocks[stock]['Close'].astype(float)
    df_stocks[stock]['Open']=df_stocks[stock]['Open'].astype(float)
    df_stocks[stock]['High']=df_stocks[stock]['High'].astype(float)
    df_stocks[stock]['Low']=df_stocks[stock]['Low'].astype(float)
    #set the date column as the new index
    df_stocks[stock]=df_stocks[stock].set_index("Date")
    print("sucessfully updated "+stock+" index and column dtype")
    

In [17]:
for stock_name in stock_ticker:
    change_column(stock_name)

sucessfully updated Apple index and column dtype
sucessfully updated Google index and column dtype
sucessfully updated Ebay index and column dtype
sucessfully updated Pfizer index and column dtype


Lets see if this worked....

In [18]:
df_stocks["Apple"].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 505 entries, 2019-10-01 to 2021-09-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Stock   505 non-null    object 
 1   Open    505 non-null    float64
 2   High    505 non-null    float64
 3   Low     505 non-null    float64
 4   Close   505 non-null    float64
dtypes: float64(4), object(1)
memory usage: 23.7+ KB


Lets see what our dataframe looks like now..

In [19]:
df_stocks["Apple"].head(50)

,Stock,Open,High,Low,Close
Date,,,,,
2019-10-01,AAPL,56.267502,57.055000,56.049999,56.147499
2019-10-02,AAPL,55.764999,55.895000,54.482498,54.740002
2019-10-03,AAPL,54.607498,55.240002,53.782501,55.205002
2019-10-04,AAPL,56.410000,56.872501,55.972500,56.752499
2019-10-07,AAPL,56.567501,57.482498,56.459999,56.764999
2019-10-08,AAPL,56.455002,57.014999,56.082500,56.099998
2019-10-09,AAPL,56.757500,56.947498,56.410000,56.757500
2019-10-10,AAPL,56.982498,57.610001,56.825001,57.522499
2019-10-11,AAPL,58.237499,59.410000,58.077499,59.052502


Great! All thats left to do now is save the dataframes as csv files.

In [20]:
def df_dump(stock):
    file_name="%s.csv"%(stock)
    out_path=dir_raw/file_name
    df_stocks[stock].to_csv(out_path)
    print("Successfully saved data for %s stock" % stock)

In [21]:
for stock_name in stock_ticker:
    df_dump(stock_name)

Successfully saved data for Apple stock
Successfully saved data for Google stock
Successfully saved data for Ebay stock
Successfully saved data for Pfizer stock
